In [1]:
from topn_eval.MovieLens import MovieLens
from topn_eval.RecommenderMetrics import RecommenderMetrics
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import LeaveOneOut

In [2]:
ml = MovieLens()

In [3]:
print("Loading movie ratings...")
data = ml.loadMovieLensLatestSmall()

Loading movie ratings...


In [4]:
trainSet, testSet = train_test_split(data, test_size=.25, random_state=1)

In [5]:
algo = SVD(random_state=10)
algo.fit(trainSet)

In [6]:
predictions = algo.test(testSet)

In [7]:
print("RMSE: ", accuracy.rmse(predictions, verbose=False))
print("MAE: ", accuracy.mae(predictions, verbose=False))

RMSE:  0.9033701087151802
MAE:  0.6977882196132263


In [8]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8953  0.8950  0.8974  0.8953  0.9027  0.8972  0.0029  
MAE (testset)     0.6899  0.6892  0.6919  0.6900  0.6927  0.6907  0.0013  
Fit time          6.93    7.33    7.10    7.09    7.35    7.16    0.16    
Test time         0.17    0.16    0.20    0.21    0.21    0.19    0.02    


{'test_rmse': array([0.89532384, 0.89496855, 0.89739704, 0.89533128, 0.90273029]),
 'test_mae': array([0.68985597, 0.68918583, 0.69190124, 0.6899874 , 0.69268032]),
 'fit_time': (6.9293532371521,
  7.333629131317139,
  7.098981142044067,
  7.0936503410339355,
  7.349006175994873),
 'test_time': (0.1715073585510254,
  0.16414594650268555,
  0.198868989944458,
  0.2052769660949707,
  0.20741581916809082)}

The calculations for top N evaluation metrics below have been adapted from Frank Kane's udemy course on building recommender systems. https://www.udemy.com/building-recommender-systems-with-machine-learning-and-ai/

In [9]:
print("\nEvaluating top-10 recommendations...")
LOOCV = LeaveOneOut(n_splits=1, random_state=1)


Evaluating top-10 recommendations...


In [10]:
for trainSet, testSet in LOOCV.split(data):
    print("Computing recommendations with leave-one-out...")

    # Train model without left-out ratings
    algo.fit(trainSet)

    # Predicts ratings for left-out ratings only
    print("Predict ratings for left-out set...")
    leftOutPredictions = algo.test(testSet)

    # Build predictions for all ratings not in the training set
    print("Predict all missing ratings...")
    bigTestSet = trainSet.build_anti_testset()
    allPredictions = algo.test(bigTestSet)

    # Compute top 10 recs for each user
    print("Compute top 10 recs per user...")
    topNPredicted = RecommenderMetrics.GetTopN(allPredictions, n=10)

    # See how often we recommended a movie the user actually rated
    # Number of successful hits from top N / # of users
    print("\nHit Rate: ", RecommenderMetrics.HitRate(topNPredicted, leftOutPredictions))

    # Compute ARHR
    # The higher up in the list the hit is, the better the ARHR (cumulative value of 1/rank over divided by # of users)
    print("\nARHR (Average Reciprocal Hit Rank): ", RecommenderMetrics.AverageReciprocalHitRank(topNPredicted, leftOutPredictions))

    # See how often we recommended a movie the user actually liked
    # Disregard hits that had less than a threshold for rating
    print("\ncHR (Cumulative Hit Rate, rating >= 4): ", RecommenderMetrics.CumulativeHitRate(topNPredicted, leftOutPredictions, 4.0))

    # Break down hit rate by rating value
    print("\nrHR (Hit Rate by Rating value): ")
    RecommenderMetrics.RatingHitRate(topNPredicted, leftOutPredictions)

Computing recommendations with leave-one-out...
Predict ratings for left-out set...
Predict all missing ratings...
Compute top 10 recs per user...

Hit Rate:  0.029806259314456036

ARHR (Average Reciprocal Hit Rank):  0.0111560570576964

cHR (Cumulative Hit Rate, rating >= 4):  0.04960835509138381

rHR (Hit Rate by Rating value): 
3.5 0.017241379310344827
4.0 0.0425531914893617
4.5 0.020833333333333332
5.0 0.06802721088435375
